In [1]:
#Baseline model:
#TF-IDF + Logistic

#so far: without Augmentation

In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import KFold
from sklearn import metrics


In [3]:
df_train = pd.read_csv('train_paired_cleaned.csv')
df_train

,Unnamed: 0,more_toxic_text,less_toxic_text,toxicity_more_toxic,toxicity_less_toxic,labels_more_toxic,labels_less_toxic
0,0,To enjoy frustration and helplessness of Obama...,Well the issues of gender oppression forced la...,0.166667,0.100000,0,0
1,1,Trudeau Liberals national climate deal to have...,The stoners don t really care since they can g...,0.800000,0.200000,1,0
2,2,Actually Carl Hillary s crimes lies and sociop...,If you are doing business in a public forum an...,0.300000,0.166667,0,0
3,3,Liberals are stupid unskilled immigrants are s...,Most Catholics have no alternative to the Novu...,0.883333,0.200000,1,0
4,4,As usual Trump is right on Think about it this...,There s a couple of pretty big differences and...,0.200000,0.111111,0,0
...,...,...,...,...,...,...,...
99995,99995,Having recently returned from a few of the so ...,Bob Hawley and Connie S The apparently deleted...,0.166667,0.000000,0,0
99996,99996,So nothing concrete to protest they just do no...,And Obama appointed his basketball buddy We sa...,0.166667,0.166667,0,0
99997,99997,Stand behind your President guys that is how a...,To continue Walter you write In regard to tran...,0.500000,0.166667,0,0
99998,99998,I saw it on the news the night BEFORE his spee...,Of course he isn t Mariota but he s not Jonsen...,0.166667,0.166667,0,0


In [4]:
df_test = pd.read_csv('val_cleaned.csv')
df_test

,Unnamed: 0,less_toxic_text,more_toxic_text
0,0,You are either deluded about the nature of you...,Russians bring us Vodka What a stereotype No s...
1,1,Hey what is wrong with you greeks all standing...,fuck you you son of a bitch you fucking bastar...
2,2,Damn fellas Paul Pressey came to the NBA after...,Impotent leader Jimbo you promised to do somet...
3,3,I will not tolerate this racism from all of yo...,Protest The article about jews and progrom fro...
4,4,we can tell that you have a great uprising FP S,Sorry really mean it now Go fuck yourself FAGG...
...,...,...,...
15405,15405,Perfect example gar the Jacobite Church and th...,If that book was a joke why the Greeks killed ...
15406,15406,Caminosoto you don t know what you re talking ...,And next time actually first create the voting...
15407,15407,Hi Tbhotch can you revisit this FLC when you g...,You fucked with the wrong person
15408,15408,YOU IDIOT WHY U DELETE MY PAGE,Wikipedia is useless You idiots defie the laws...


In [5]:
df_train['less_toxic_text'] = df_train['less_toxic_text'].apply(lambda x: ' '.join([w for w in str(x).lower().split()]))
df_train['more_toxic_text'] = df_train['less_toxic_text'].apply(lambda x: ' '.join([w for w in str(x).lower().split()]))

In [6]:
df_test['less_toxic_text'] = df_test['less_toxic_text'].apply(lambda x: ' '.join([w for w in str(x).lower().split()]))
df_test['more_toxic_text'] = df_test['less_toxic_text'].apply(lambda x: ' '.join([w for w in str(x).lower().split()]))

In [7]:
less_toxic_df_train = pd.DataFrame()
less_toxic_df_train['text'] = df_train['less_toxic_text']
less_toxic_df_train['target'] = len(df_train['less_toxic_text'])*[0]

In [8]:
more_toxic_df_train = pd.DataFrame()
more_toxic_df_train['text'] = df_train['more_toxic_text']
more_toxic_df_train['target'] = len(df_train['more_toxic_text'])*[1]

In [9]:
final_df_train = pd.concat([less_toxic_df_train, more_toxic_df_train])
final_df_train = final_df_train.sample(frac=0.1, random_state=2).reset_index(drop=True)
final_df_train

,text,target
0,seems that you don t remember tricky dicky nix...,1
1,ah the liberal cabal is at it again don t know...,0
2,don t even try to have a sensible conversation...,1
3,proof,1
4,one more quote suitable to the day as democrac...,1
...,...,...
19995,piano2 maybe dancing with the stars or he coul...,0
19996,those same fisherman also leave the state with...,0
19997,any lifestyle choice we make should be one we ...,0
19998,sure mhlobo wam makhosi koza mcebisi jonas lin...,0


In [10]:
less_toxic_df_test = pd.DataFrame()
less_toxic_df_test['text'] = df_test['less_toxic_text']
less_toxic_df_test['target'] = len(df_test['less_toxic_text'])*[0]

In [11]:
more_toxic_df_test = pd.DataFrame()
more_toxic_df_test['text'] = df_test['more_toxic_text']
more_toxic_df_test['target'] = len(df_test['more_toxic_text'])*[1]

In [12]:
final_df_test = pd.concat([less_toxic_df_test, more_toxic_df_test])
final_df_test = final_df_test.sample(frac=0.1, random_state=2).reset_index(drop=True)
final_df_test

,text,target
0,images for deletion are you out of your mind w...,1
1,know that salivo sucks,1
2,hello clown rust never sleeps 76 223 246 87,0
3,british isles please stop adding nonsense to t...,0
4,nangparbat just to let you know this aint me s...,0
...,...,...
3077,not really requesting an unblock because i m n...,1
3078,andranikpasha your nickname expalins who are y...,0
3079,mfnickster i am not sure if you have ever had ...,0
3080,don t blame me you know you re wrong m man and...,1


In [13]:
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))

tfidf_vec.fit_transform(final_df_train['text'].values.tolist() + final_df_test['text'].values.tolist())

tfidf_train = tfidf_vec.transform(final_df_train['text'].values.tolist())
tfidf_test = tfidf_vec.transform(final_df_test['text'].values.tolist())

In [14]:
y = final_df_train['target'].values

In [15]:
def runModel(X_train, y_train, X_val, X_test):
    model = LogisticRegression(C=5., solver='sag')
    model.fit(X_train, y_train)
    pred_y_val = model.predict(X_val)
    pred_y_test = model.predict(X_test)
    return pred_y_val, pred_y_test, model

In [16]:
print('Building model')

kf = KFold(n_splits=5, shuffle=True, random_state=2023)
models=[]
fold=0
pred_test = 0
acc_scores=[]
preds_train = np.zeros([final_df_train.shape[0]])

for train_index, val_index in kf.split(final_df_train):
    print('-'*50)
    print('Fold: ', fold)
    print('-'*50)
    
    X_train, X_val = tfidf_train[train_index], tfidf_train[val_index]
    
    y_train, y_val = y[train_index], y[val_index]
    
    print('Training...')
    pred_y_val, pred_y_test, model = runModel(X_train, y_train, X_val, tfidf_test)
    
    models.append(model)
    pred_test = pred_test + pred_y_test
    preds_train[val_index] = pred_y_val
    
    acc = metrics.accuracy_score(y_val, pred_y_val)
    acc_scores.append(acc)
    print(f'Accuracy for fold {fold}: ', acc)
    fold+=1
    

Building model
--------------------------------------------------
Fold:  0
--------------------------------------------------
Training...
Accuracy for fold 0:  0.46725
--------------------------------------------------
Fold:  1
--------------------------------------------------
Training...
Accuracy for fold 1:  0.46625
--------------------------------------------------
Fold:  2
--------------------------------------------------
Training...
Accuracy for fold 2:  0.45875
--------------------------------------------------
Fold:  3
--------------------------------------------------
Training...
Accuracy for fold 3:  0.477
--------------------------------------------------
Fold:  4
--------------------------------------------------
Training...
Accuracy for fold 4:  0.45875


In [17]:
preds = [round(y_t) for y_t in pred_test/5]
print('Accuracy on test set: ', metrics.accuracy_score(final_df_test.target, preds))

Accuracy on test set:  0.4990266060999351
